In [1]:
import time
import astroquery.heasarc
from astropy.coordinates import SkyCoord
import astropy.units as u
from astroquery.jplhorizons import Horizons
import numpy as np
import pandas as pd
from collections import defaultdict
from astropy.time import Time

Idea of how this code works

1. We obtain Jupiter's position in the sky (ephemerides). We will be using a search radius of 8 degrees (due to FOV of INTEGRAL, fully coded). I found that Jupiter's position does not significantly change (more than 8 degrees) within less than 2 months. For safety, we will obtain Jupiter's mean monthly position.
2. Using the mean monthly position of Jupiter over a certain time period (1 year, 2 years, ...), we will search for all the ScWs within 8 degrees of this position. No time filtering yet. We will group by mean monthly position of Jupiter.
Though we already filter for pointing ScWs only.
3. Then, we have N groups of ScWs, each corresponding to a certain RA and DEC (within 8 degrees). These only contain Jupiter if it is in the FOV at that precise time, i.e. that month. So filtering will be done by taking ScWs only within corresponding month (aka the month corresponding to the mean position as found in step 1).
4. Finally, we will have N groups of ScWs, each containg Jupiter, for each month in the given interval. Some may be empty.

STEP 1 - obtaining Jupiter's position

In [2]:
# First, define the time interval we are interested in (> 1 month)
epochs = {'start':'2003-01-01', 'stop':'2004-01-01', 'step':'31d'}

# Get Jupiter's monthly position in the given time interval (not using mean as step of 31 days)
jupiter = Horizons(id='599', location='@0',epochs=epochs) 
eph = jupiter.ephemerides()

ra_values = eph['RA']
dec_values = eph['DEC']
datetime_values = eph['datetime_jd']
print(f"Jupiter's monthly position between {epochs['start']} and {epochs['stop']}: \n")
print(eph[['datetime_str', 'RA', 'DEC']])

monthly_intervals = []
for i in range(len(datetime_values) - 1):  # Get the intervals between successive months
    start_time = Time(datetime_values[i], format='jd').mjd
    end_time = Time(datetime_values[i + 1], format='jd').mjd
    monthly_intervals.append((start_time, end_time))
print(f"Monthly intervals based on Jupiter's positions:\n {monthly_intervals}")

# If we want to use mean:
# df = eph.to_pandas()
# df['datetime_str'] = pd.to_datetime(df['datetime_str'])
# monthly_means = df.groupby(df['datetime_str'].dt.strftime('%Y-%m'))[['RA', 'DEC']].mean()
# for month, row in monthly_means.iterrows():
    # print(f"Month: {month}, Mean RA: {row['RA']:.2f} deg, Mean DEC: {row['DEC']:.2f} deg")

Jupiter's monthly position between 2003-01-01 and 2004-01-01: 

   datetime_str       RA      DEC   
       ---           deg      deg   
----------------- --------- --------
2003-Jan-01 00:00 133.13461 18.23402
2003-Feb-01 00:00 135.64353 17.59249
2003-Mar-04 00:00  138.1248 16.92216
2003-Apr-04 00:00 140.57899 16.22478
2003-May-05 00:00 143.00678 15.50206
2003-Jun-05 00:00 145.40891  14.7557
2003-Jul-06 00:00 147.78632 13.98736
2003-Aug-06 00:00    150.14 13.19866
2003-Sep-06 00:00 152.47101 12.39117
2003-Oct-07 00:00 154.78053 11.56645
2003-Nov-07 00:00 157.06978 10.72599
2003-Dec-08 00:00 159.34001  9.87127
Monthly intervals based on Jupiter's positions:
 [(52640.0, 52671.0), (52671.0, 52702.0), (52702.0, 52733.0), (52733.0, 52764.0), (52764.0, 52795.0), (52795.0, 52826.0), (52826.0, 52857.0), (52857.0, 52888.0), (52888.0, 52919.0), (52919.0, 52950.0), (52950.0, 52981.0)]


STEP 2 - obtaining ScWs

In [3]:
Heasarc = astroquery.heasarc.Heasarc()

def get_scw_list(ra_obj, dec_obj, radius):
    R = Heasarc.query_region(
            position = SkyCoord(ra_obj, dec_obj, unit='deg'),
            radius = f"{radius} deg",
            catalog = 'intscw',
            good_isgri = ">1000",
        )
    R.sort('scw_id') 
    return R['scw_id'], R['scw_ver'], R['start_date'], R['end_date']

assert astroquery.__version__ >= '0.4.2.dev6611'
assert 'isdc' in astroquery.heasarc.Conf.server.cfgtype

In [4]:
scw_ids = []
scw_versions = []
scw_start_times = []
scw_end_times = []

for row in eph:
    ra = float(row['RA'])
    dec = float(row['DEC'])
    radius = 8.
    scw_id, scw_ver, scw_start, scw_end = get_scw_list(ra, dec, radius)
    scw_ids.append(scw_id) # Using append means we get a list of lists, grouped by Jupiter's monthly position
    scw_versions.append(scw_ver)
    scw_start_times.append(scw_start)
    scw_end_times.append(scw_end)

# Filter by type (pointing type)
scw_ids_filtered = []
scw_versions_filtered = []
scw_start_times_filtered = []
scw_end_times_filtered = []

for scw_sublist, ver_sublist, start_sublist, end_sublist in zip(scw_ids, scw_versions, scw_start_times, scw_end_times):
    
    mask = [int(scw) % 10 == 0 for scw in scw_sublist] # divisible by 10 (ends with a 0 <=> pointing type)
    
    filtered_scws = [scw_sublist[i] for i in range(len(scw_sublist)) if mask[i]]
    filtered_versions = [ver_sublist[i] for i in range(len(ver_sublist)) if mask[i]]
    filtered_start_times = [start_sublist[i] for i in range(len(start_sublist)) if mask[i]]
    filtered_end_times = [end_sublist[i] for i in range(len(end_sublist)) if mask[i]]
    
    scw_ids_filtered.append(filtered_scws)
    scw_versions_filtered.append(filtered_versions)
    scw_start_times_filtered.append(filtered_start_times)
    scw_end_times_filtered.append(filtered_end_times)

scw_ids = scw_ids_filtered
scw_versions = scw_versions_filtered
scw_start_times = scw_start_times_filtered
scw_end_times = scw_end_times_filtered

print("Found a total of", np.sum([len(scw_id) for scw_id in scw_ids]), " pointing ScWs.")
print("Number of ScWs found per month: \n")
for i, scw_sublist in enumerate(scw_ids):
    print(f"Month {i+1} - # of ScWs found: {len(scw_sublist)}")

Found a total of 3474  pointing ScWs.
Number of ScWs found per month: 

Month 1 - # of ScWs found: 377
Month 2 - # of ScWs found: 358
Month 3 - # of ScWs found: 312
Month 4 - # of ScWs found: 417
Month 5 - # of ScWs found: 354
Month 6 - # of ScWs found: 306
Month 7 - # of ScWs found: 290
Month 8 - # of ScWs found: 302
Month 9 - # of ScWs found: 321
Month 10 - # of ScWs found: 159
Month 11 - # of ScWs found: 147
Month 12 - # of ScWs found: 131


STEP 3 - filtering by time

In [ ]:
scw_ids_filtered = []
scw_versions_filtered = []
scw_start_times_filtered = []
scw_end_times_filtered = []

for month_index, (scw_sublist, ver_sublist, start_sublist, end_sublist) in enumerate(zip(scw_ids, scw_versions, scw_start_times, scw_end_times)):
    filtered_scws_month = []
    filtered_versions_month = []
    filtered_start_times_month = []
    filtered_end_times_month = []

    if month_index < len(monthly_intervals):
        month_start, month_end = monthly_intervals[month_index]
    else:
        month_start, month_end = monthly_intervals[-1] 

    for scw_id, start_time, end_time in zip(scw_sublist, start_sublist, end_sublist):
            if month_start <= start_time <= month_end: 
                filtered_scws_month.append(scw_sublist)
                filtered_versions_month.append(ver_sublist[scw_sublist.index(scw_id)])
                filtered_start_times_month.append(start_sublist)
                filtered_end_times_month.append(end_sublist) 
    
    scw_ids_filtered.append(filtered_scws_month)
    scw_versions_filtered.append(filtered_versions_month)
    scw_start_times_filtered.append(filtered_start_times_month)
    scw_end_times_filtered.append(filtered_end_times_month)

print("Found a total of", np.sum([len(scw_id) for scw_id in scw_ids_filtered]), "matching ScWs.")
print("Number of matching ScWs found per month: \n")
for i, scw_sublist in enumerate(scw_ids_filtered):
    if len(scw_sublist)!=0:
        print(f"Month {i+1} - # of ScWs found: {len(scw_sublist)}")

Found a total of 1796 matching ScWs.
Number of matching ScWs found per month: 

Month 1 - # of ScWs found: 0
Month 2 - # of ScWs found: 0
Month 3 - # of ScWs found: 0
Month 4 - # of ScWs found: 0
Month 5 - # of ScWs found: 0
Month 6 - # of ScWs found: 0
Month 7 - # of ScWs found: 0
Month 8 - # of ScWs found: 0
Month 9 - # of ScWs found: 0
Month 10 - # of ScWs found: 0
Month 11 - # of ScWs found: 0
Month 12 - # of ScWs found: 185
Month 13 - # of ScWs found: 46
Month 14 - # of ScWs found: 0
Month 15 - # of ScWs found: 0
Month 16 - # of ScWs found: 0
Month 17 - # of ScWs found: 32
Month 18 - # of ScWs found: 4
Month 19 - # of ScWs found: 0
Month 20 - # of ScWs found: 0
Month 21 - # of ScWs found: 0
Month 22 - # of ScWs found: 0
Month 23 - # of ScWs found: 0
Month 24 - # of ScWs found: 0
Month 25 - # of ScWs found: 0
Month 26 - # of ScWs found: 2
Month 27 - # of ScWs found: 0
Month 28 - # of ScWs found: 0
Month 29 - # of ScWs found: 0
Month 30 - # of ScWs found: 0
Month 31 - # of ScWs foun

We can save these ScWs to a file, with Jupiter's expected position.

In [ ]:
# with open(f"{epochs['start']}_{epochs['stop']}.txt", "w+") as f:
    # for (ids, vers, starts, ends) in zip(scw_ids, scw_versions, scw_start_times, scw_end_times):
        # f.write("{0}, {1}, {2}, {3}\n".format(ids, vers, starts, ends))

# Flatten the lists
flat_ids = [item for sublist in scw_ids_filtered for item in sublist]
flat_versions = [item for sublist in scw_versions_filtered for item in sublist]
flat_start_times = [item for sublist in scw_start_times_filtered for item in sublist]
flat_end_times = [item for sublist in scw_end_times_filtered for item in sublist]

with open(f"../data/{epochs['start']}_{epochs['stop']}.txt", "w") as f:
    f.write("ID, Version, Start Time, End Time, Jupiter RA, Jupiter DEC\n")
    for id, version, start, end, ra, dec in zip(flat_ids, flat_versions, flat_start_times, flat_end_times, ra_values, dec_values):
        f.write(f"{id}, {version}, {start}, {end}, {ra}, {dec}\n")